<a href="https://colab.research.google.com/github/nov05/Google-Colaboratory/blob/master/20241129_finetune_bert_solution_2_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* changed by nov05 on 2024-11-28  
* Udacity AWS MLE Nanodegree (ND189)  
  Course 4, 3.15 Excercise: Fine-Tuning BERT    
* local env `conda activate cuda_py310` with cuda enabled  
* [CoLA dataset on KaggleHub](https://www.kaggle.com/datasets/krazy47/cola-the-corpus-of-linguistic-acceptability)  
  The Corpus of Linguistic Acceptability   

In [8]:
!mkdir -p cola_public
!mkdir -p cola_public/raw
!wget https://raw.githubusercontent.com/nov05/udacity-CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter/refs/heads/main/cd0387_common_model_arch_types_fine_tuning/cola_public/raw/in_domain_train.tsv -O cola_public/raw/in_domain_train.tsv

--2024-11-29 13:12:54--  https://raw.githubusercontent.com/nov05/udacity-CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter/refs/heads/main/cd0387_common_model_arch_types_fine_tuning/cola_public/raw/in_domain_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 428578 (419K) [text/plain]
Saving to: ‘cola_public/raw/in_domain_train.tsv’

cola_public/raw/in_ 100%[===================>] 418.53K  --.-KB/s    in 0.03s   

2024-11-29 13:12:54 (12.1 MB/s) - ‘cola_public/raw/in_domain_train.tsv’ saved [428578/428578]



## Solution: Fine-tune BERT model

In [9]:
import os
# import sys
# import json
from tqdm import tqdm
import wandb
import numpy as np
import pandas as pd
import torch
from torch.optim import AdamW
# import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer # type: ignore
from transformers import get_linear_schedule_with_warmup # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.utils.class_weight import compute_class_weight # type: ignore

## log training process with W&B if uncommented
# os.environ['WANDB_MODE'] = 'disabled'

In [10]:
class Config:
    def __init__(self):
        self.wandb = True
        self.device = torch.device('cpu')
        self.max_len = 64 ## this is the max length of the sentence
        self.epochs = 15
        self.batch_size = 64
        self.opt_lr = 2e-5

config = Config()
config.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"👉 Running on device type: {config.device}")

👉 Running on device type: cuda:0


In [11]:
df = pd.read_csv(
   r"/content/cola_public/raw/in_domain_train.tsv",
   sep="\t",
   header=None,
   usecols=[1, 3],
   names=["label", "sentence"],
)
sentences = df.sentence.values
labels = df.label.values
print(df.shape)
df.sample(3)

(8551, 2)


,label,sentence
1507,1,Do you believe that somebody was looking for s...
4444,0,Americans have paying income tax ever since 1913.
6507,1,"Almost every cat likes mice, but Felix doesn't."


In [12]:
!mkdir -p data
!mkdir -p data/cola
train_df, test_df = train_test_split(df, stratify=labels)
train_df.to_csv(r"data/cola/train.csv", index=False)
test_df.to_csv(r"data/cola/test.csv", index=False)

In [13]:
print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat==labels_flat) / len(labels_flat)


def get_train_data_loader(batch_size):
    dataset = pd.read_csv(os.path.join("data", "cola", "train.csv"))
    sentences = dataset.sentence.values
    labels = dataset.label.values

    input_ids = []
    for sent in sentences:
        encoded_sent = tokenizer.encode(sent, add_special_tokens=True)
        input_ids.append(encoded_sent)

    # pad shorter sentences
    input_ids_padded = []
    for id in input_ids:
        while len(id) < config.max_len:
            id.append(0)
        input_ids_padded.append(id)
    input_ids = input_ids_padded

    # mask; 0: added, 1: otherwise
    attention_masks = []
    # For each sentence...
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)

    # convert to PyTorch data types.
    train_inputs = torch.tensor(input_ids)
    train_labels = torch.tensor(labels)
    train_masks = torch.tensor(attention_masks)

    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    return train_dataloader


def get_test_data_loader(test_batch_size):
    dataset = pd.read_csv(os.path.join("data", "cola", "test.csv"))
    sentences = dataset.sentence.values
    labels = dataset.label.values

    input_ids = []
    for sent in sentences:
        encoded_sent = tokenizer.encode(sent, add_special_tokens=True)
        input_ids.append(encoded_sent)

    # pad shorter sentences
    input_ids_padded = []
    for id in input_ids:
        while len(id)<config.max_len:
            id.append(0)
        input_ids_padded.append(id)
    input_ids = input_ids_padded

    # mask; 0: added, 1: otherwise
    attention_masks = []
    # For each sentence...
    for sent in input_ids:
        att_mask = [int(token_id>0) for token_id in sent]
        attention_masks.append(att_mask)

    # convert to PyTorch data types.
    train_inputs = torch.tensor(input_ids)
    train_labels = torch.tensor(labels)
    train_masks = torch.tensor(attention_masks)

    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=test_batch_size)

    return train_dataloader


def train():
    train_loader = get_train_data_loader(config.batch_size)
    test_loader = get_test_data_loader(config.batch_size)
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab.
        num_labels=2,  # The number of output labels--2 for binary classification.
        output_attentions=False,  # Whether the model returns attentions weights
        output_hidden_states=False,  # Whether the model returns all hidden-states
    )
    model = model.to(config.device)
    optimizer = AdamW(model.parameters(), lr=config.opt_lr)
    total_steps = 0

    for epoch in tqdm(range(config.epochs)):
        print(f"👉 Train Epoch {epoch}:")
        loss_epoch = 0
        model.train()
        for step, batch in enumerate(train_loader):
            total_steps += 1
            b_input_ids = batch[0].to(config.device)
            b_input_mask = batch[1].to(config.device)
            b_labels = batch[2].to(config.device)
            model.zero_grad()

            outputs = model(
                b_input_ids,  ## Shape: (batch_size, sequence_length)
                token_type_ids=None, ## Shape: (batch_size, sequence_length)
                attention_mask=b_input_mask, ## Shape: (batch_size, sequence_length)
                labels=b_labels) ## Shape: (batch_size,)

            loss = outputs.loss  ## same with outputs[0]
            wandb.log({"train_loss": loss.item()}, step=total_steps)
            loss_epoch += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
            optimizer.step()
            if step%10==0:
                print(
                    f"Step {total_steps}: "
                    f"[{step*len(batch[0])}/{len(train_loader.sampler)} "
                    f"({(100.0*step/len(train_loader)):.0f}%)] "
                    f"Loss: {loss.item():.6f}"
                )
        wandb.log({"train_loss_epoch": loss_epoch/config.batch_size}, step=total_steps)
        eval_accuracy = test(model, test_loader)
        wandb.log({f"eval_accuracy_epoch (%)": eval_accuracy*100}, step=total_steps)
    return model


def test(model, test_loader):
    model.eval()
    eval_accuracy_steps = 0
    total_steps = 0
    with torch.no_grad():
        for batch in test_loader:
            total_steps += 1
            b_input_ids = batch[0].to(config.device)
            b_input_mask = batch[1].to(config.device)
            b_labels = batch[2].to(config.device)
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
            logits = outputs.logits.detach().cpu().numpy()  ## differs from the original
            label_ids = b_labels.to("cpu").numpy()
            eval_accuracy_steps += flat_accuracy(logits, label_ids)
    eval_accuracy = eval_accuracy_steps / total_steps
    print("\n🟢 Test Accuracy (%): ", eval_accuracy*100)
    return eval_accuracy

Loading BERT tokenizer...


In [14]:
%%time
wandb.init(
    # set the wandb project where this run will be logged
    project="udacity-awsmle-bert-cola",
    config=config
)
train()
## 15 epochs, 16:49 mins

train_loss,▆█▅▄▇▆▄▇▇▇▄▆█▅▆▅▅▅▆▃▂▄▅▆▂▄▅▃▆▆▂▂▄▅▃▄▂▁▁▄
train_loss_epoch,▁
train_loss,0.42
train_loss_epoch,0.82879


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/15 [00:00<?, ?it/s]

👉 Train Epoch 0:
Step 1: [0/6413 (0%)] Loss: 1.015233
Step 11: [640/6413 (10%)] Loss: 0.596859
Step 21: [1280/6413 (20%)] Loss: 0.577084
Step 31: [1920/6413 (30%)] Loss: 0.561362
Step 41: [2560/6413 (40%)] Loss: 0.511488
Step 51: [3200/6413 (50%)] Loss: 0.428192
Step 61: [3840/6413 (59%)] Loss: 0.448093
Step 71: [4480/6413 (69%)] Loss: 0.438618
Step 81: [5120/6413 (79%)] Loss: 0.460993
Step 91: [5760/6413 (89%)] Loss: 0.498080
Step 101: [1300/6413 (99%)] Loss: 0.379479


  7%|▋         | 1/15 [01:05<15:23, 65.94s/it]


🟢 Test Accuracy (%):  79.05472285067874
👉 Train Epoch 1:
Step 102: [0/6413 (0%)] Loss: 0.340742
Step 112: [640/6413 (10%)] Loss: 0.342892
Step 122: [1280/6413 (20%)] Loss: 0.435111
Step 132: [1920/6413 (30%)] Loss: 0.422198
Step 142: [2560/6413 (40%)] Loss: 0.389161
Step 152: [3200/6413 (50%)] Loss: 0.468284
Step 162: [3840/6413 (59%)] Loss: 0.437216
Step 172: [4480/6413 (69%)] Loss: 0.310340
Step 182: [5120/6413 (79%)] Loss: 0.437729
Step 192: [5760/6413 (89%)] Loss: 0.377489
Step 202: [1300/6413 (99%)] Loss: 0.269135


...

 93%|█████████▎| 14/15 [15:16<01:05, 65.40s/it]


🟢 Test Accuracy (%):  80.89649321266968
👉 Train Epoch 14:
Step 1415: [0/6413 (0%)] Loss: 0.002144
Step 1425: [640/6413 (10%)] Loss: 0.006059
Step 1435: [1280/6413 (20%)] Loss: 0.001450
Step 1445: [1920/6413 (30%)] Loss: 0.068424
Step 1455: [2560/6413 (40%)] Loss: 0.000472
Step 1465: [3200/6413 (50%)] Loss: 0.000764
Step 1475: [3840/6413 (59%)] Loss: 0.030084
Step 1485: [4480/6413 (69%)] Loss: 0.098110
Step 1495: [5120/6413 (79%)] Loss: 0.000306
Step 1505: [5760/6413 (89%)] Loss: 0.001544
Step 1515: [1300/6413 (99%)] Loss: 0.000132


100%|██████████| 15/15 [16:21<00:00, 65.46s/it]


🟢 Test Accuracy (%):  81.44089366515837
CPU times: user 16min 18s, sys: 1.89 s, total: 16min 20s
Wall time: 16min 32s


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
wandb.finish()

eval_accuracy_epoch (%),▁▆▆▆▆▅▄▆▆▆█▅█▆▇
train_loss,██▅▇▃▄▃▃▃▃▁▂▂▁▁▂▁▂▁▁▂▁▁▂▁▁▁▂▂▁▁▁▁▁▁▁▂▁▁▁
train_loss_epoch,█▆▄▃▂▂▂▂▂▁▁▁▁▁▁
eval_accuracy_epoch (%),81.44089
train_loss,0.00013
train_loss_epoch,0.02814


```python
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermediate_act_fn): GELUActivation()
          )
          (output): BertOutput(
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (pooler): BertPooler(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (activation): Tanh()
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
)
```

* [Unfreezing all layers of BERT giving good results than freezing and adding custom Forward layer for Fine-Tuning](https://www.reddit.com/r/MLQuestions/comments/1d07qlz/unfreezing_all_layers_of_bert_giving_good_results/)   
* [I just can't fine tune BERT over 40% accuracy for text-classification task](https://www.reddit.com/r/MachineLearning/comments/1bx5r8r/d_i_just_cant_fine_tune_bert_over_40_accuracy_for/)  